Importing libraries

In [71]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [72]:
SEED_VAL = 0
tf.__version__
#tf.random.set_seed(SEED_VAL)

'2.17.0'

In [73]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
yeast = fetch_ucirepo(id=110) 

X = yeast.data.features.to_numpy() 
y_raw = yeast.data.targets.to_numpy()

### Converting y to a 1D array and Encode it.

In [74]:
y = y_raw.ravel()
print(set(y))
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y = le.fit_transform(y)

{'NUC', 'POX', 'ERL', 'CYT', 'EXC', 'VAC', 'ME3', 'ME2', 'MIT', 'ME1'}


## Encode y column

In [75]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(sparse_output=False)
encoded_categories = onehotencoder.fit_transform(y.reshape(-1, 1))
print(encoded_categories.shape)
y = encoded_categories
y.shape

(1484, 10)


(1484, 10)

## Splitting the dataset into the Training and Test sets

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [77]:
print(X.shape)
y_test.shape

(1484, 8)


(371, 10)

## Feature Scaling
we scale the features so they're in the same range

In [78]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the ANN

### Initializing the ANN

In [79]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [80]:
ann.add(tf.keras.layers.Dense(units=16, activation='relu'))

### Adding the second hidden layer

In [81]:
ann.add(tf.keras.layers.Dense(units=8, activation='relu'))

### Adding the output layer

In [82]:
ann.add(tf.keras.layers.Dense(units=10, activation='softmax'))

## Training The ANN

### Compiling the ANN

In [83]:
ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [84]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100, shuffle=False)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0900 - loss: 2.3428
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2337 - loss: 2.2153 
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3119 - loss: 2.0935 
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2998 - loss: 1.9503
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3014 - loss: 1.8006 
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3108 - loss: 1.6827
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3362 - loss: 1.6021 
Epoch 8/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4052 - loss: 1.5432 
Epoch 9/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4643 - loss: 1.4957 
Epoch 10/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4813 - loss: 1.4533
Epoch 11/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4995 - loss: 1.4129 
Epoch 12/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - ac

## Model Predictions and Evaluations

In [89]:
# Evaluate the model
loss_train, accuracy_train = ann.evaluate(X_train, y_train)
loss, accuracy = ann.evaluate(X_test, y_test)
print(f"Train Accuracy: {accuracy_train:.2f}")
print(f"Test Accuracy: {accuracy:.2f}")

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6207 - loss: 0.9515 
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5604 - loss: 1.1127 
Train Accuracy: 0.63
Test Accuracy: 0.58


### Predicting Insample test results

In [85]:
y_pred_ins = ann.predict(X_train)
y_pred_ins = (y_pred_ins > 0.5).astype("int")

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [86]:
# Getting the accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_train, y_pred_ins)
#print(cm)
accuracy_score(y_train, y_pred_ins)

ValueError: multilabel-indicator is not supported

### Out-Sample Prediction

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5).astype("int")
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
#print(cm)
accuracy_score(y_test, y_pred)